## Input

### User Input

- Input Voltage $V_{in}$
- Output Power $P_o=250$
- Output Voltage, $V_o = 115$ volts
- Frequency $f=47$
- Efficiency $\eta=95\%$
- Operating flux density $B_{ac}$
- Temperature rise goal, $T_r=30^oC$
- Window utilization, $K_u=0.4$
- Core Material = $Silicon M6X$
- Lamination Number = $EI-150$
- Current Density $J=250 [amps/cm^2]$
- Regulation $\alpha = 5$ $\%$

### Constants 

- Window utilization $K_u$

In [2]:
# Loading libraries
import pandas as pd
import numpy as np 

In [3]:
Frequency = 47 # Hz
Temperature_rise_goal = 30 # degree Celcius
Output_power = 250 # watts
Efficiency = 95 # %
Input_voltage = 115
Output_voltage = 115 # volts


## Apparent Power $P_t$

Calculating the transformer apparent power, $P_t$

$$ P_t=P_o(\dfrac{1}{\eta} + 1), [watts]$$

$$P_t=250[(1/0.95)+1]$$

$$P_t=513, [watts]$$

In [4]:
Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
print(Apparent_power)

513.1578947368421


## Calculation of Area Product

$$J=\dfrac{P_t (10^4)}{K_f K_u B_{ac} f A_p} $$

$$A_p=\dfrac{P_t (10^4)}{K_f K_u B_{ac} f J} $$

$A_p$ is in $cm^4$ 

For initial case :

$$K_f = 4.44$$
$$K_u = 0.4 $$
$$B_{ac} = 1.6$$
$$f= 47$$
$$J=250$$

In [5]:
K_f = 4.44 
K_u = 0.4
B_ac = 1.6
J = 250
Area_product = (Apparent_power*(10**4))/(K_f * K_u * B_ac * J * Frequency)
Area_product *= 10**4
print(Area_product) # mm^4

1536916.2555612721


In [6]:
lamination_data = pd.read_csv('EI-Laminations.csv')
lamination_data['Type']

0     SB1R
1     SB3R
2       17
3       74
4      12A
5       74
6       23
7       30
8       31
9       15
10      33
11       3
12      16
13       6
14      43
15       8
16      8B
17      8C
18    I180
19    I120
20     S14
21     S12
22     S15
23      16
24      43
25    S197
26    S222
Name: Type, dtype: object

In [7]:
selected_lamination = lamination_data[lamination_data['Type'] == '12A']
selected_lamination

,Type,A,B,C,D,E,Tongue,Winding-width,Winding-length,K-ratio,Remarks,Fig-No
4,12A,15.87,47.62,38.68,7.93,7.93,15.87,7.945,22.82,2877.308763,-,1


Find stack and cost for 60% of Area product

Increase stack by 5mm and find the cost until Area product < 1.4 * Area product


In [8]:
selected_lamination_k = selected_lamination['K-ratio']
selected_lamination_k

4    2877.308763
Name: K-ratio, dtype: float64

In [9]:
selected_lamination_tongue = selected_lamination['Tongue']
selected_lamination_tongue

4    15.87
Name: Tongue, dtype: float64

if stack is greater than 5 * Tongue then discard that stack.

In [10]:
stack = .6 * Area_product/ selected_lamination_k 
# if we print stack we will get  0 stack in series dtype 
# for assesing the value we will use .max() 
stack = stack.max()

# now for getting stack stack in multiple of 5 
if stack%5 == 0:
  print(stack)
else:
  stack = stack - stack % 5
  print(stack)
  print(stack + 5)

# let's check the stack condition using tongue 
print('Stack:  ' + str(stack))
print('Tongue: ' + str(selected_lamination_tongue.max()))

320.0
325.0
Stack:  320.0
Tongue: 15.87


## Primary turns

$$A_c = tongue*stack$$



In [11]:
tongue = selected_lamination['Tongue']
A_c = tongue * stack 

$$N_p = \dfrac{ V_{in}}{K_f B_{ac} f A_c}$$

In [12]:
primary_turns = (Input_voltage)/(K_f* B_ac* Frequency* A_c)
print(primary_turns)

4    0.000068
Name: Tongue, dtype: float64


## From step 7 to everything

**Input Current $I_{in}$**

$$I_{in} = P_o $$

**Calculate the primary bare wire area**

**Select the wire from Wire Table**

**Mean Turn Length**


**Turns per layer (TPL)_p**

TPL = wl/(diameter of wire) with insulation

**Number of Layers Primary**

$$Number of turns primary / Turns per layer primary$$

**Bobbin_thicknes**

bobbin_thickness is a user defined input

let's take bobbin_thickness = 1.5

**Built**

$$ Built = \dfrac{bobbinthickness + Number of layers primary}{Diameter of wire primary(insulation  enamel)}$$

- this diameter is with insulation

**SWG of wire gauge**

**MTL (Mean turn length)**

$$ MTL = 2*(tongue + stack + 2 * built) $$

*Length of primary wire = MTL * (number of priamry turns)*

**Primary Resistence**



Resistance of primary = Resistivity of Cu * length of primary / Bare primary area

**Primary of Copper  Loss**

primary Cu loss = $I_{primary}^2$ * (Priamary Resistence)

## Secondary 

**Calculate secondary turns**

Secondary turns (Ns) = Primary_turns * secondary voltage* (1+regulation/100)/ primary voltage

**Secondary bare wire area**

Secondary bare wire area = secondary current ($I_s$) / Current Density ($J$)

**From sec bare area, select secondary SWG**

**Turns per layer secondary**

Turns per layer secondary = wl/ (diameter of wire secondary)

**Number of layers secondary**

Number of layers secondary = secondary turns ($N_s$) / Turns per layer secondary

**Built secondary**

Built secondary = Number of layer secondary + insulation_thickness

**Insulation thickness is user defined with default value 0.2mm**

**Mean Turn Length Secondary**

Mean turn Length seoncary ($MLT_s$) = 2 *(tongue + stack + 4 * built_primary + 2 * built_secondary)

**Length of Wire secondary**

Length of Wire Secondary = $MLT_{secondary}$ * Secondary_turns

**Secondary Resistence ($R_s$)**

Secondary Resistence = $\rho$ * (length of secondary) / (bare area secondary)

$\rho$ - resistivity of Copper (*Cu*)

**Secondary Copper Loss**

Secondary Cu loss = $I_{secondary}^2$ * $R_{secondary}$

**Total Cu loss**j

Calculate total Cu loss = Primary Cu loss + Secondary Cu loss

 **Core Loss**

 core loss = (Core loss factor) * Weight (Wt) of core

- core loss factor = 0.000557 * $f^a * f^b$
  - $f$ - frequency
  - a = 1.68
  - b = 1.86

**Weight of Core**

Wt of core = density of core * volume of core

density of core = density of iron * 0.97

**Volume of core is to be calculated from the EI lamination diagram**


Volume of core = stack *[ B * C-2 * ww * wl ]


**Watts per unit area**

Watts per unit area (Ψ) = Total Loss / Total Surface Area

**Temperature rise**

Temperature rise ($T_r$) = 450 *  $Ψ^{0.826}$

**Total surface area**

total surface area = 2 * [B * C + B * (stack + 2 * total_built) + C * (stack + 2 * total_built)]

Now,

We need to check *Winding width* > *total built* 

If the above condition is not true, the design won't be phisically possible to manufacture 




## Cost 




Cost = Wt of Cu * Rate of Cu + Wt of iron * rate of iron 

Note : Wt of Core is nothing but wt of iron

**Rate of Cu: 950 Rs/Kg**

**Rate of Iron = 250/Kg**

## Cost Function

cost = 